In [1]:
!python --version

Python 3.12.4


In [2]:
from constants import OPENAI_API_KEY

In [3]:
!pip show langchain-openai | grep "Version:"

Version: 0.2.9


In [4]:
import os

In [5]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [7]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [12]:
vectorstore = FAISS.from_texts(
    [
        "harrison worked at kensho",
        "harrison likes spicy food",
        "harrison is from Ohio"
    ], 
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [13]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [14]:
chain.invoke({"question": "where did harrison work", "language": "italian"})

'Harrison ha lavorato a Kensho.'

In [11]:
chain.invoke({"question": "where did harrison work", "language": "tamil"})

'ஹாரிசன் கென்ஷோவில் வேலை செய்தான்.'

In [15]:
chain.invoke({"question": "where is harrison from?", "language": "tamil"})

'ஹாரிசன் ஓஹியோவில் இருந்து வந்தவர்.'